
# How to Handle Large Text Inputs with Longformer and Hugging Face Transformers

https://www.kdnuggets.com/how-to-handle-large-text-inputs-with-longformer-and-hugging-face-transformers

Download dataset and split (the code is correct but the dataset has an error, all the labels are 0! Same problem as 009)

In [11]:
from datasets import load_dataset

dataset = load_dataset("imdb", split="train[:1000]+test[:1000]")

train_test_split = dataset.train_test_split(test_size=0.5)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [10]:
train_dataset["label"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


Tokenise

In [2]:
from transformers import LongformerTokenizer

longformer_model= "allenai/longformer-base-4096"
tokenizer = LongformerTokenizer.from_pretrained(longformer_model)

def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Prepare model for fine-tuning

In [3]:
from transformers import DataCollatorWithPadding, LongformerForSequenceClassification

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = LongformerForSequenceClassification.from_pretrained(longformer_model, num_labels=2)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training arguments

In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

c:\Users\TristramArmour\anaconda3\envs\learning\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/500 [00:00<?, ?it/s]

Initializing global attention on CLS token...


{'loss': 0.0131, 'grad_norm': 0.002323734574019909, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 4.823603740078397e-05, 'eval_runtime': 71.9683, 'eval_samples_per_second': 13.895, 'eval_steps_per_second': 6.948, 'epoch': 1.0}
{'train_runtime': 325.9972, 'train_samples_per_second': 3.068, 'train_steps_per_second': 1.534, 'train_loss': 0.013065990447998046, 'epoch': 1.0}


TrainOutput(global_step=500, training_loss=0.013065990447998046, metrics={'train_runtime': 325.9972, 'train_samples_per_second': 3.068, 'train_steps_per_second': 1.534, 'total_flos': 328425805824000.0, 'train_loss': 0.013065990447998046, 'epoch': 1.0})

In [5]:
results = trainer.evaluate()

  0%|          | 0/500 [00:00<?, ?it/s]

Evaluate model

In [6]:
import torch

long_review = "This was very scary horror movie. I loved it. The acting was great and the special effects were amazing. I would definitely recommend this movie to my friends."

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

inputs = tokenizer(long_review, return_tensors="pt", max_length=512, padding='max_length', truncation=True).to(device)

In [8]:
model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = logits.argmax(-1)

label = "positive" if prediction.item() == 1 else "negative"
print(label)

negative
